In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [3]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history = ChatMessageHistory()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

chain = prompt | chat 

chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history")

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


loader = PyPDFLoader("LLM_Scenario.pdf")
pages = loader.load_and_split()


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(pages)

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever(k=4)

In [8]:
input = "What should i do to get a housing loan?"

response = chain_with_history.invoke(
    {"input": input,
     "context": retriever.invoke(input)},
    {"configurable": {"session_id": "unused"}},
)
response

Parent run 4c6a6a3d-0345-4916-877c-bd50ce25925d not found for run f980cfe5-fe05-408e-bc77-6148b797c442. Treating as a root run.


AIMessage(content='To apply for a housing loan, you can complete the request online using the link provided: https://forms.smartbank.lk/housing/housing. Additionally, you\'ll need to calculate your eligible loan amount using the "Loan Calculator" provided on the website. You\'ll also need your Smart Bank account number (if you have one) and your financial details, including your income and expenses. If you have further questions, you can contact the Manager at your nearest Smart Branch or call the call centre at 1234 for more information.', response_metadata={'token_usage': {'completion_tokens': 107, 'prompt_tokens': 503, 'total_tokens': 610}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b953e4de39', 'finish_reason': 'stop', 'logprobs': None}, id='run-9dd548fc-28c8-4df1-9596-d9e874b83bd3-0')